# Decision Tree


**Definition-** A decision tree is a non-parametric supervised learning algorithm, which is utilized for both classification and regression tasks. It has a hierarchical, tree structure, which consists of a root node, branches, internal nodes and leaf nodes.

**Dataset**

* Fromat: each row is an example.
* The last column is the label.
* The first two columns are features.
* If you want you can add more features & examples.
* Interesting note: 2nd and 5th examples have the same features, but different labels.



In [ ]:
# defining our dataset
training_data = [
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Mango'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

testing_data = [
    ['Green', 3, 'Mango'],
    ['Yellow', 3, 'Mango'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

# column label these are used only to print the tree
header = ['Color', 'Diameter', 'Fruit']

**Required Functions**

In [ ]:
# Returns the set of unique values for a column in a dataset.
def unique_vals(rows, col):
   # set initializing using list comprehension
  return set([row[col] for row in rows])


# Returns the set of number of each type of example in a dataset.
def class_counts(rows):
  # a dictionary of label(fruit name) -> count
  counts = {}
  for row in rows:
    # in our dataset format, the label(fruit name) is always the last column
    label = row[-1]
    if label not in counts:
      counts[label] = 0
    counts[label] += 1
  return counts


# Return true if number is either int or float
def is_numeric(value):
  return isinstance(value, (int, float))


**A Question is used to partition a dataset**

This class just records a 'column number' (e.g. 0 for Color) and a 'column value' (e.g. Green). The 'match' method is used to compare the feature value in an example to the feature value stored in the question.

In [ ]:
class Question:
  def __init__(self, column, value):
    self.column = column
    self.value = value

  # Compares the feature value in an example to the feature value in this question.
  def match(self, example):
    val = example[self.column]
    if is_numeric(val):
      return val >= self.value
    else:
      return val == self.value

  # This is just a helper method to print the question in a readable format.
  def __repr__(self):
    condition = '=='
    if is_numeric(self.value):
      condition = '>='
    return "Is %s %s %s?" % (header[self.column], condition, str(self.value))

**Partition a dataset**

For each row in the dataset, check if it matches the question. If so, add it to 'true rows', otherwise, add it to 'false rows'.

In [ ]:
# Returns tuple of lists(true rows, false rows)
def partition(rows, question):
  true_rows, false_rows = [], []
  for row in rows:
    if question.match(row):
      true_rows.append(row)
    else:
      false_rows.append(row)
    return true_rows, false_rows

**Gini Impurity**

Calculate the Gini Impurity for a list of rows.

In [ ]:
def gini(rows):
  counts = class_counts(rows)
  impurity = 1
  for lbl in counts:
    prob_of_lbl = counts[lbl] / float(len(rows))
    impurity -= prob_of_lbl**2
  return impurity

**Information Gain**

The uncertainty of the starting node, minus the weighted impurity of two child nodes.

In [ ]:
def info_gain(left, right, current_uncertainty):
  p = float(len(left)) / (len(left) + len(right))
  return current_uncertainty - p * gini(left) - (1-p) * gini(right)


**Find best split**

Find the best question to ask by iterating over every feature / value and calculating the information gain.

In [ ]:
def find_best_split(rows):
  # keep track of the best information gain
  best_gain = 0
  # keep train of the feature / value that produced it
  best_question = None
  current_uncertainty = gini(rows)
  # number of columns
  n_features = len(rows[0]) - 1

  for col in range(n_features):
    # for each feature
    values = set([row[col] for row in rows]) # unique values in the column
    for val in values:
      # for each row
      question = Question(col, val)
      # try splitting the dataset
      true_rows, false_rows = partition(rows, question)

      # skip this split if it doesn't divide the dataset
      if len(true_rows) == 0 or len(false_rows) == 0:
        continue  
      
      # calculate the information gain from this split
      gain = info_gain(true_row, false_rows, current_uncertainty)

      if gain >= best_gain:
        best_gain, best_question = gain, question

    return best_gain, best_question


**Leaf**

A leaf node classifies data.

In [ ]:
class Leaf:
  # This holds a dictionary of class (e.g. "Mango") -> number of times
  # it appears in the rows from the training data that reach this leaf.
  def __init__(self, rows):
    self.predictions = class_counts(rows)

**Decision Node**

A Decision Node asks a question.

In [ ]:
class Decision_Node:
  # This holds a reference to the question, and to the two child nodes.
  def __init__(self, question, true_branch, false_branch):
    self.question = question
    self.true_branch = true_branch
    self.false_branch = false_branch


**Builds the tree**

In [ ]:
def build_tree(rows):
  # Try partitioning the dataset on each of the unique attribute,
  # calculate the information gain, and return the question that produces the highest gain.
  gain, question = find_best_split(rows)

  # Base case: no further info gain
  # Since we can ask no further questions, we'll return a Leaf.
  if gain == 0:
    return Leaf(rows)

  # If we reach here, we have found a useful feature / value to partiotion on.
  true_rows, false_rows = partition(rows, question)

  # Recursively build the true branch.
  true_branch = build_tree(true_rows)

  # Return a Question node.
  # This records the best feature / value to ask at this point, as well as the branches to follow depending on the answer.
  return Decision_Node(question, true_branch, false_branch)


**Printing tree**

In [ ]:
def print_tree(node, spacing = ""):
  # Base case: We've reached a Leaf
  if isinstance(node, Leaf):
    print (spacing + "Predict", node.predictions)
    return
  
  # Print the question at this node
  print (spacing + str(node.question))

  # Call this function recursively on the true branch
  print (spacing + '--> True:')
  print_tree(node.true_branch, spacing + " ")

  # Call this function recursively on the true branch
  print (spacing + '--> False:')
  print_tree(node.false_branch, spacing + " ")


In [ ]:
def classify(row, node):
  # Base case: we've reached a Leaf
  if isinstance(node, Leaf):
    return node.predictions
  
  # Decide whether to follow the true-branch or the false-branch.
  # Compare the feature / value stored in the node, to the example we're considering.
  if node.question.match(row):
    return classify(row, node.true_branch)
  else:
    return classify(row, node.false_branch)

**Print the predictions at a Leaf.**

In [ ]:
def print_leaf(counts):
  total = sum(counts.values()) * 1.0
  probs = {}
  for lbl in counts.keys():
    probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
  return probs

**Running our program**

In [ ]:
# if __name__ == '__Main__':
my_tree = build_tree(training_data)
print_tree(my_tree)

  # Evaluate
for row in testing_data:
  print("Actual: %s. Predicted: %s" %(row[-1], print_leaf(classify(row, my_tree))))


Predict {'Mango': 2, 'Grape': 2, 'Lemon': 1}
Actual: Mango. Predicted: {'Mango': '40%', 'Grape': '40%', 'Lemon': '20%'}
Actual: Mango. Predicted: {'Mango': '40%', 'Grape': '40%', 'Lemon': '20%'}
Actual: Grape. Predicted: {'Mango': '40%', 'Grape': '40%', 'Lemon': '20%'}
Actual: Grape. Predicted: {'Mango': '40%', 'Grape': '40%', 'Lemon': '20%'}
Actual: Lemon. Predicted: {'Mango': '40%', 'Grape': '40%', 'Lemon': '20%'}
